# Pro-Football 12-lead Resting Electrocardiogram Database

**Data collected**

The data were gathered from La Liga, Spain, from professional football players. 

Resting in supine position, each participantʼs 12-lead electrocardiogram (ECG) was captured with General Electrics (GE) USB-CAM 14 for a duration of 10 s at 500 Hz using the GE CardioSoft software.

## Extracting waveform from GE Muse XML

https://github.com/PierreElias/IntroECG/tree/master/1-Waveform%20Extraction

## Extracting waveform from PDF